# Convert Polar Stereographic Files to Lat/Lon Files

This notebook provides a workflow for converting polar stereographic netcdf files to lat/lon coordinates. To get the data into the proper format, we needed to do the following:
 - utilize a template file as a base for modifying netcdf files of the existing mask arrays. The template file includes all proper GIS attributes.
 - python script reads the Geotiff file and creates a netcdf file with the x,y,lat,lon coordinate dimensions and variables

Scripts were originally created by John Truesdale and updated by Teagan King.
 
To Do:
------
 - recreate template file with a polar processing python notebook
 - set up loop through files and write out processed files
 - use the standard ESMF mapping procedure to go from the projected stereographic polar mask grids to our regular gridded data from CESM. Use Steve's python code and John's mapping file commands to do this
 - keep timestamp in eg hours since date... perhaps simulation start date, use datetime python library

Some notes from John on methods for regridding:
-----------------------------------------------
 - used QGIS to assign a standard stereographic coordinate and create a GeoTiff file version of one of the Polar jpg plots
 - Giving a standard location to each jpeg pixel basically consisted of finding similar features at the pixel level (the tip of an island, the most inset part of a promenent bay) between our polar jpegs and a map that is already georeferenced and has known locations for those pixels. If you choose 3-10 pixels in common you can create a linear regression that will map out all the rest of the pixels on your jpeg.
 - Once the GIS application was able to calculate the transform to go from pixel to a standard coordinate system, I saved all that information in a GeoTiff file.
 - The polar projection jpegs on which the climatenet masks are drawn were created from python matplotlib and you can grab the coordinate information from matplotlib; this was checked with QGIS
 - Because the LLNL polar jpegs are a projected coordinate, the underlying unit in a stereographic projection is meters.  The x and y variables on the GeoTiff and converted netcdf file contain meter offsets of every pixel (row,col) of the ar_mask array with respect to one of the standard south pole stereographic coordinate systems.
 -   When you look at the square projected polar image you see that the longitude lines converge at the pole and latitudes are a set of nested circles.  When you are describing this grid in lat/lon coordinates it is known as a curvilinear grid where each pixel (array location) requires a unique lat/lon pair to specify its position on a regular grid. A straight line along any row or column of the jpeg raster (or ar_mask array) will intersect different lat lon values for every pixel.  The coordinate information for our rectangular ar_mask array therefore contains lat and lon variables that are two dimensions and describe the entire grid of 1152x1152 points with unique lat/lon values for each pixel (array location) of ar_mask. The standard netcdf way of denoting a curvilinear coordinate is by creating the dimensions that define the size of the ar_mask array (x,y), adding lat/lon variables that are each dimensioned (x,y) containing the lat/lon coordinates of that point, and finally adding metadata to the ar_mask array noting that the coordinates for this variable are not the dimension variables x,y but the lat/lon variables.

In [39]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import os
from rasterio.warp import transform
import urllib.request
import xarray as xr
import glob

### Generate template file (only needs to be done once)

In [29]:
# Read the data
input_path='/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-copy-sav1.tif'

da = xr.open_rasterio(input_path)
yval=da['y']
ryval=np.flip(yval)
# Compute the lon/lat coordinates with rasterio.warp.transform
ny, nx = len(da['y']), len(da['x'])
x, y = np.meshgrid(da['x'], ryval)
# Rasterio works with 1D arrays
lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
                     x.flatten(), y.flatten())
lon = np.asarray(lon).reshape((ny, nx))
lat = np.asarray(lat).reshape((ny, nx))
da.coords['lon'] = (('y', 'x'), lon)
da.coords['lat'] = (('y', 'x'), lat)

/glade/scratch/tking/ipykernel_95971/966493801.py:4: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  da = xr.open_rasterio(input_path)


In [6]:
da.to_netcdf(path='/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-sav1-rev-latlon.nc')
# use for just antarctic

## loop through mask files to generate converted mask files

In [140]:
qa1_antarctic = glob.glob('/glade/u/home/tking/work/cgnet/QA_xml/h5/qa*/antarctic/netcdfs/data*')
for index in range(len(qa1_antarctic)):
    
    # read in mask file
    qa1_aa_ds = xr.open_dataset(qa1_antarctic[index])

    # read in temp file for use in notebook
    temp = xr.open_dataset('/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-sav1-rev-latlon.nc')
    
    # rename the dimensions of the original ar_masks variable from phony_0/1 to y and x respectively
    qa1_aa_ds = qa1_aa_ds.rename({'phony_dim_0':'y','phony_dim_1':'x'})

    # append the new x,y,lat,lon variables to mask using the GeoTiff temp file
    temp['ar_masks'] = qa1_aa_ds.ar_masks
    
    # add all the coordinate attributes to ar_mask
    temp['ar_masks'] = temp['ar_masks'].assign_attrs({'transform' : temp.__xarray_dataarray_variable__.transform,
                                                      'crs': temp.__xarray_dataarray_variable__.crs,
                                                      'res': temp.__xarray_dataarray_variable__.res,
                                                      'is_tiled': temp.__xarray_dataarray_variable__.is_tiled,
                                                      'nodatavals': temp.__xarray_dataarray_variable__.nodatavals,
                                                      'scales': temp.__xarray_dataarray_variable__.scales,
                                                      'offsets': temp.__xarray_dataarray_variable__.offsets,
                                                      'AREA_OR_POINT': temp.__xarray_dataarray_variable__.AREA_OR_POINT})
    filename = qa1_antarctic[index].split('/')[-1]
    temp.to_netcdf(path='/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/'+filename)
    print('made netcdf {}'.format(filename))

made netcdf data-2001-01-01-00-2.nc
made netcdf data-2001-03-01-00-2.nc
made netcdf data-2001-05-14-00-2_1.nc
made netcdf data-2002-02-22-00-2_1.nc
made netcdf data-2001-12-03-00-2.nc
made netcdf data-2001-11-04-00-2.nc
made netcdf data-2001-10-06-00-2.nc
made netcdf data-2001-07-23-00-2.nc
made netcdf data-2001-12-23-00-2.nc
made netcdf data-2001-07-14-00-2.nc
made netcdf data-2002-01-30-00-2.nc
made netcdf data-2001-07-23-00-2_2.nc
made netcdf data-2001-06-23-00-2.nc
made netcdf data-2001-05-14-00-2.nc
made netcdf data-2001-02-17-00-2.nc
made netcdf data-2001-03-01-00-2_2.nc
made netcdf data-2001-07-23-00-2_3.nc
made netcdf data-2001-09-09-00-2.nc
made netcdf data-2001-02-14-00-2_1.nc
made netcdf data-2001-07-14-00-2_1.nc
made netcdf data-2001-06-30-00-2.nc
made netcdf data-2001-03-15-00-2.nc
made netcdf data-2002-01-09-00-2.nc
made netcdf data-2001-09-07-00-2.nc
made netcdf data-2001-04-05-00-2.nc
made netcdf data-2001-03-03-00-2.nc
made netcdf data-2001-12-05-00-2.nc
made netcdf da

# PART 2:

### use the standard ESMF mapping procedure to go from the projected stereographic polar mask grids to our regular gridded data from CESM. Use Steve's python code and John's mapping file commands to do this

In [35]:
# We just want to do this to get masks and state information on two different grids

In [ ]:
# need new mapping file to do this, then run through Steve's routine